# Collecting and Analying Data
---
## Football Matchs

## Processing the data with Pandas

In [ ]:
import pandas as pd

### Reading datasets (CSV files)
#### Keeping the key collumns as indexes

In [ ]:
def read_matches(year: int) -> pd.DataFrame:
    return pd.read_csv(f'../../datasets/exercise-1/jogos_{year}.csv', index_col=['data', 'id_clube1', 'id_clube2'])

def read_clubs() -> pd.DataFrame:
    return pd.read_csv('../../datasets/exercise-1/clubes.csv', index_col='id_clube')

def read_states() -> pd.DataFrame:
    return pd.read_csv('../../datasets/exercise-1/estados.csv', index_col='id_estado')

def read_arenas() -> pd.DataFrame:
    return pd.read_csv('../../datasets/exercise-1/arenas.csv', index_col='id_arena')

### Getting a data sample

In [ ]:
df_sample = read_matches(2017)
print('Shape: ', df_sample.shape)

df_sample.sample(frac=0.1, random_state=1)

### Getting a basic stats about numeric fields in the data frame

In [ ]:
df_sample.describe()

### Concating all matchs in an unique data frame

In [ ]:
def read_all_matches() -> pd.DataFrame:
     return pd.concat([read_matches(2013), read_matches(2014), read_matches(2015), read_matches(2016), read_matches(2017)])

df_matches = read_all_matches()
print('New Shape: ', df_matches.shape)

#### Basic stats after data concat

In [ ]:
df_matches.describe()

#### Data sample after concat

In [ ]:
df_matches.sample(frac=0.01, random_state=1)

### Joining data in an unique data frame (Denormalization)

In [ ]:
df_arenas = read_arenas()
df_clubs = read_clubs()
df_states = read_states()

df_matches = (read_all_matches()
                .join(df_arenas, how='left', on='id_arena')
                .join(df_clubs, how='left', on='id_clube1')#.rename(columns={'desc_clube': 'desc_clube1'})
                .join(df_clubs, how='left', on='id_clube2', lsuffix='1', rsuffix='2')#.rename(columns={'desc_clube': 'desc_clube2'})
                .join(df_clubs, how='left', on='id_clube_vencedor').rename(columns={'desc_clube': 'desc_clube_vencedor'})
                .join(df_states, how='left', on='id_estado_clube1')
                .join(df_states, how='left', on='id_estado_clube2', lsuffix='1', rsuffix='2')
                .join(df_states, how='left', on='id_estado_vencedor').rename(columns={'sigla_estado': 'sigla_estado_vencedor', 'desc_estado': 'desc_estado_vencedor'}))

df_matches.sample(frac=0.005)

## Exploring data
At this point we already have the ready to work. And now we will explore it.

## Answering some questions

### 1. Which club did more goals in all seasons?

In [ ]:
def sum_goals_by_club(df: pd.DataFrame) -> pd.DataFrame:
    df_club1 = (df.reset_index()[['desc_clube1', 'qtd_gols_clube1']]
                .rename(columns={'desc_clube1': 'desc_clube', 'qtd_gols_clube1': 'qtd_gols_clube'}))
            
    df_club2 = (df.reset_index()[['desc_clube2', 'qtd_gols_clube2']]
                .rename(columns={'desc_clube2': 'desc_clube', 'qtd_gols_clube2': 'qtd_gols_clube'}))

    df_clubs = pd.concat([df_club1, df_club2])
    df_sum_goals = df_clubs.groupby('desc_clube')['qtd_gols_clube'].sum().reset_index()
    return df_sum_goals.sort_values(by='qtd_gols_clube', ascending=False)

# Getting top 5
sum_goals_by_club(df_matches).head(5)

### 2. Which club did more goals in 2015 season?

In [ ]:
def filter_by_year(df: pd.DataFrame, year) -> pd.DataFrame:
    df_r = df_matches.reset_index()
    return df_r[df_r['data'].str.slice(start=6, stop=10, step=1) == str(year)]

df_15 = filter_by_year(df_matches, 2015)

# Getting top 5
sum_goals_by_club(df_15).head(5)

### 3. Which match had more goals in all the seasons?

In [ ]:
df_matches["total_gols"] = df_matches["qtd_gols_clube1"] + df_matches["qtd_gols_clube2"]

# Getting top 5
df_matches.sort_values(by="total_gols", ascending=False).head(5)


### 4. Which was the greatest victory in all the seasons (Considering the diference of goals between the winner and the loser team)

In [ ]:
import numpy as np

df_matches['dif_gols'] = np.power(df_matches['qtd_gols_clube1'] - df_matches['qtd_gols_clube2'], 2) # To prevent negative values we can also use method "abs() - absolute values" rather than to use power and sqrt
df_matches['dif_gols'] = np.sqrt(df_matches['dif_gols'])

df_matches.sort_values(by='dif_gols', ascending=False).reset_index()[['desc_clube_vencedor', 'dif_gols']].head(5)

In [ ]:
df_without_idx = df_matches.reset_index(drop=True) # Reseting indexes and dropping columns
df_without_idx.sample(frac=0.01)

### 5. Which state was the greatest winner in 2016 (Match winner)

In [ ]:
df_16 = filter_by_year(df_matches, 2016)
df_16 = df_16[df_16['id_estado_vencedor'] != 0]
df_16.groupby('desc_estado_vencedor')['desc_estado_vencedor'].count().sort_values(ascending=False).head(1)

---
#### Correlation
https://towardsdatascience.com/why-feature-correlation-matters-a-lot-847e8ba439c4

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_heatmap(train):
    correlations = train.corr()

    fig, ax = plt.subplots(figsize=(10,10))
    sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f',
                square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
    plt.show()
    
correlation_heatmap(df_matches)